Data Preprocessing



In [83]:
"""
This notebook is used to preprocess the data gathered from different Data sources and to extract the most useful features.

The data sources used are:
- The street traffic csv file. 
- Temperature and Occupation status of intensive care beds["taux_occupation_sae"] [covid-19 related]
- Key dates concerning curfew ["couvre feu"] and lockdown ["confinement"] status
- Key dates regarding holidays

The output features after-preprocessing are:
  [
    Date et heure de comptage,Débit horaire,Taux d'occupation,Etat trafic,Libelle noeud amont,Libelle noeud aval,
    date,weekday,weekofyear,month,year,time,tmin,tmax,mean_taux_occupation_past_week,taux_occupation_past_week,
    mean_debit_horaire_past_week,debit_horaire_past_week,taux_occupation_sae,confin_0,confin_1,confin_2,couvrefeu,ferie
  ]

The notebook is to be used 3 times for each street. After each execution, a new CSV file name <streetname>_edited.csv 
is created in the folder "datasets".

"""

'\nThis notebook is used to preprocess the data gathered from different Data sources and to extract the most useful features.\n\nThe data sources used are:\n- The street traffic csv file. \n- Temperature and Occupation status of intensive care beds["taux_occupation_sae"] [covid-19 related]\n- Key dates concerning curfew ["couvre feu"] and lockdown ["confinement"] status\n- Key dates regarding holidays\n\nThe output features after-preprocessing are:\n  [\n    Date et heure de comptage,Débit horaire,Taux d\'occupation,Etat trafic,Libelle noeud amont,Libelle noeud aval,\n    date,weekday,weekofyear,month,year,time,tmin,tmax,mean_taux_occupation_past_week,taux_occupation_past_week,\n    mean_debit_horaire_past_week,debit_horaire_past_week,taux_occupation_sae,confin_0,confin_1,confin_2,couvrefeu,ferie\n  ]\n\nThe notebook is to be used 3 times for each street. After each execution, a new CSV file name <streetname>_edited.csv \nis created in the folder "datasets".\n\n'

In [84]:
import pandas as pd
import requests 
import json
from datetime import datetime , timedelta
key = "biXmYSG60H0IuxAdhaZbvGPNDTBPwzoX"
import time

In [87]:
#file_name = 'datasets/washington_11_dec.csv'
#file_name = 'datasets/convention_11_dec.csv'
file_name = 'datasets/sts_11_dec.csv'


if (file_name == 'datasets/washington_11_dec.csv' ):
  save_name = 'datasets/washington_edited.csv'
  amont = "Av_Champs_Elysees-Washington"
  aval = "Av_Champs_Elysees-Berri"
  print("Street selected: {}".format(file_name) )

elif (file_name == 'datasets/convention_11_dec.csv'):
  save_name = 'datasets/convention_edited.csv'
  amont = "Lecourbe-Convention"
  aval = "Convention-Blomet"
  print("Street selected: {}".format(file_name) )

elif (file_name == 'datasets/sts_11_dec.csv'):
  save_name = 'datasets/sts_edited.csv'
  amont = "Sts_Peres-Voltaire"
  aval = "Sts_Peres-Universite"
  print("Street selected: {}".format(file_name) )

else:
    print("Wrong street selected!!" )

Street selected: datasets/sts_11_dec.csv


Date Features

In [88]:
df = pd.read_csv(file_name, sep = ";")
df = df[["Date et heure de comptage","Débit horaire","Taux d'occupation","Etat trafic","Libelle noeud amont","Libelle noeud aval"]]
def filter(df,amont,aval):
    df = df[df["Libelle noeud amont"]==amont]
    df = df[df["Libelle noeud aval"]==aval]
    return df

def set_date(df):
    df["Date et heure de comptage"] = pd.to_datetime(df["Date et heure de comptage"],format='%Y-%m-%dT%H:%M:%S%z')
    return df
df = filter(df,amont,aval)
df = set_date(df)
df = df.sort_values(by = "Date et heure de comptage")
df = df.interpolate() #remove Nan by interpolation
df = df.reset_index()
df["date"] = [d.date() for d in df["Date et heure de comptage"]]
df["weekday"] = [d.weekday() for d in df["Date et heure de comptage"]]
df["weekofyear"] = [d.weekofyear for d in df["Date et heure de comptage"]]
df["month"] = [d.month for d in df["Date et heure de comptage"]]
df["year"] = [d.year for d in df["Date et heure de comptage"]]
df["time"] = [d.time() for d in df["Date et heure de comptage"]]


Weather Data (Hourly and Daily)

In [89]:
#  ADD HOURLY DATA
"""
count = 0
test = True
start = df.loc[0,"date"]
fin = df.loc[df.index[-1],"date"]
hd = []
while test : 
  print(count)
  url = "https://api.meteostat.net/v2/stations/hourly?station=07157&start="+start.strftime('%Y-%m-%d')+"&end="+(start+timedelta(9)).strftime('%Y-%m-%d')
  r = requests.get(url,headers={'x-api-key':key})
  d = json.loads(r.text)
  if len(d['data']) <10 : 
    print("Problem")
  else :
    hd = hd + d['data']
  start = start + timedelta(9)
  if start > fin : 
    test = False
  count = count +1
  time.sleep(3)
hd = pd.DataFrame.from_records(hd)
hd = hd[["time","temp","dwpt","rhum"]]
hd["time"] = pd.to_datetime(hd["time"],format='%Y-%m-%d %H:%M:%S')
hd = hd.sort_values(by = "time")
hd = hd.drop_duplicates().reset_index(drop = True)
hd = hd.interpolate()
hd = hd.rename(columns = {"time":"Date et heure de comptage"})
hd["date"] = [d.date() for d in hd["Date et heure de comptage"]]
hd["time"] = [d.time() for d in hd["Date et heure de comptage"]]
del hd['Date et heure de comptage']
df = pd.merge(df,hd,on =['time','date'], how = 'left').sort_values(by = "Date et heure de comptage")

"""

'\ncount = 0\ntest = True\nstart = df.loc[0,"date"]\nfin = df.loc[df.index[-1],"date"]\nhd = []\nwhile test : \n  print(count)\n  url = "https://api.meteostat.net/v2/stations/hourly?station=07157&start="+start.strftime(\'%Y-%m-%d\')+"&end="+(start+timedelta(9)).strftime(\'%Y-%m-%d\')\n  r = requests.get(url,headers={\'x-api-key\':key})\n  d = json.loads(r.text)\n  if len(d[\'data\']) <10 : \n    print("Problem")\n  else :\n    hd = hd + d[\'data\']\n  start = start + timedelta(9)\n  if start > fin : \n    test = False\n  count = count +1\n  time.sleep(3)\nhd = pd.DataFrame.from_records(hd)\nhd = hd[["time","temp","dwpt","rhum"]]\nhd["time"] = pd.to_datetime(hd["time"],format=\'%Y-%m-%d %H:%M:%S\')\nhd = hd.sort_values(by = "time")\nhd = hd.drop_duplicates().reset_index(drop = True)\nhd = hd.interpolate()\nhd = hd.rename(columns = {"time":"Date et heure de comptage"})\nhd["date"] = [d.date() for d in hd["Date et heure de comptage"]]\nhd["time"] = [d.time() for d in hd["Date et heure de 

In [90]:
# DAILY DATA
count = 0
test = True
start = df.loc[0,"date"]
fin = df.loc[df.index[-1],"date"]
dd = []
while test : 
  print(count)
  url = "https://api.meteostat.net/v2/stations/daily?station=07157&start="+start.strftime('%Y-%m-%d')+"&end="+min(start+timedelta(369),fin).strftime('%Y-%m-%d')
  r = requests.get(url,headers={'x-api-key':key})
  d = json.loads(r.text)
  if len(d['data']) <10 : 
    print("Problem")
  else :
    dd = dd + d['data']
  start = start + timedelta(369)
  if start > fin : 
    test = False
  count = count +1
  time.sleep(3)
dd = pd.DataFrame.from_records(dd)
dd["date"] = pd.to_datetime(dd["date"],format='%Y-%m-%d')
dd = dd.sort_values(by = "date")
dd = dd[["date","tmin","tmax"]]
dd = dd.interpolate()
dd.date = dd.date.apply(lambda x : x.date())
df = pd.merge(df,dd,on ='date', how = 'left').sort_values(by = "Date et heure de comptage")

0
1


In [91]:
df["mean_taux_occupation_past_week"] = df["Taux d'occupation"].rolling(24*7).mean().shift(24*6)
df["taux_occupation_past_week"] = df["Taux d'occupation"].shift(24*7)
df["mean_debit_horaire_past_week"] = df["Débit horaire"].rolling(24*7).mean().shift(24*6)
df["debit_horaire_past_week"] = df["Débit horaire"].shift(24*7)

In [92]:
indicateur = pd.read_csv('datasets/table-indicateurs-open-data-france.csv')
indicateur.drop(["tx_incid","tx_pos","R"],axis = 1, inplace =True)
indicateur= indicateur.rename(columns ={"extract_date" : "date"})
indicateur['taux_occupation_sae'] = indicateur['taux_occupation_sae'].interpolate()
indicateur.date = pd.to_datetime(indicateur["date"],format='%Y-%m-%d')
indicateur.date = indicateur.date.apply(lambda x : x.date())
indicateur.to_csv('datasets/covid.csv')

In [93]:
covid =pd.read_csv('datasets/covid.csv')
del covid['Unnamed: 0']
covid.date = pd.to_datetime(covid["date"],format='%Y-%m-%d')
covid.date = covid.date.apply(lambda x : x.date())
df = pd.merge(df,covid,on = 'date',how = 'left')
z = df[df['taux_occupation_sae'].isna()].index[-2]
df.loc[:z,'taux_occupation_sae'] = df.loc[z+1,'taux_occupation_sae']
f = df[df['taux_occupation_sae'].isna()].index[-1]
df.loc[f:,'taux_occupation_sae'] = df.loc[f-1,'taux_occupation_sae']

# Mets à jours RETAIL and Taux d'occupation des lits de réa


confi = pd.read_csv('datasets/confinement couvre feu.csv',sep = ";")
confi = pd.concat([pd.get_dummies(confi.confinement, prefix= 'confin'),confi], axis= 1).drop(columns = ['confinement'])
confi.date = pd.to_datetime(confi["date"],format='%Y-%m-%d')
confi.date = confi.date.apply(lambda x : x.date())
df = pd.merge(df,confi,on = 'date',how = 'left') 

jf = pd.read_csv('datasets/feries.csv')
jf.nom_jour_ferie = 1
jf = jf.drop(['annee','zone'],axis = 1)
jf.date = pd.to_datetime(jf["date"],format='%Y-%m-%d')
jf.date = jf.date.apply(lambda x : x.date())
jf.rename(columns = {"nom_jour_ferie":"ferie"}, inplace = True)
df = pd.merge(df,jf,on = 'date',how = 'left') 
df.ferie = df.ferie.fillna(0)
df.ferie = df.ferie.apply(lambda x : int(x))
df=df[24*7+24*6:].reset_index(drop = True)

In [94]:
df.to_csv(save_name)
print("{} created !".format(save_name))

datasets/sts_edited.csv created !


In [95]:
df["taux_occupation_sae"][:]

0       15.2
1       15.2
2       15.2
3       15.2
4       15.2
        ... 
9377    58.3
9378    58.3
9379    58.3
9380    58.3
9381    58.3
Name: taux_occupation_sae, Length: 9382, dtype: float64